In [1]:
import os
import sys
sys.path.insert(0, "..")
import inspect
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

import wandb
from safetensors import safe_open
from tqdm import tqdm

import matplotlib.pyplot as plt
# from notebook_utils import *

from models import *
from my_datasets import *
from experiments import *
from experiments.utils.model_loader_utils import *

In [8]:
model = TheoryAutoregKStepsModel(8, 3)

In [9]:
dataset = AutoregDiamondTokensDataset(8, 32, 4.0, 10000)
dataloader = DataLoader(dataset, batch_size=4)

In [10]:
num_dones, num_hits = 0, 0
pbar = tqdm(dataloader)
for batch in pbar:
    tokens, labels = batch["tokens"], batch["labels"]
    out = model(tokens=tokens, labels=labels)
    pred = (out.logits > 0).long()
    num_dones += tokens.size(0)
    num_hits += (labels == pred).float().mean(dim=(1,2)).sum()
    acc = num_hits / num_dones
    desc = f"acc {acc:.3f}"
    pbar.set_description(desc)

acc 1.000: 100%|██████████| 2500/2500 [00:06<00:00, 404.32it/s]


In [12]:
batch["abcdefgh"]

tensor([[3, 5, 4, 0, 2, 1, 6, 7],
        [5, 4, 2, 6, 1, 3, 0, 7],
        [6, 7, 0, 1, 5, 3, 4, 2],
        [0, 3, 2, 1, 7, 4, 5, 6]])

In [21]:
(tokens[0][:,7] == 1).sum()

tensor(26)

In [ ]:
n = 16
_, res_dataset = load_model_and_dataset_from_big_grid(2*n, n)
model = TheoryAutoregKStepsModel(num_vars=n, num_steps=3)

In [ ]:
atk_dataset = KnowledgeAmnesiaDataset(res_dataset, dataset_len = 1024)
dataloader = DataLoader(atk_dataset, batch_size=3)

In [ ]:
for batch in dataloader:
    tokens = batch["tokens"]
    abcde = batch["abcde"]
    adv_labels = batch["labels"]
    break

In [ ]:
a, b, c, d, e = abcde.chunk(5, dim=-1)

In [ ]:
atk_rule = torch.cat([F.one_hot(a,n), -1000*F.one_hot(a,n)], dim=-1).view(-1,1,2*n)

In [ ]:
adv_tokens = torch.cat([atk_rule, tokens], dim=1)

In [ ]:
adv_out = model(tokens=adv_tokens)

In [ ]:
adv_pred = (adv_out.logits > 0).long()

In [ ]:
adv_pred

In [ ]:
adv_labels = batch["labels"]

In [ ]:
atk_rule = 